Perform Exploratory Data Analysis (EDA) analysis on the following:

- Summary Statistics & Missing-Value Report


In [20]:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

In [ ]:
from scripts.inspect_data import inspect_data
from scripts.clean_data import clean_data
from scripts.read_file import read_csv_file

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)


Reading csv data using pandas

In [ ]:
df = pd.read_csv('../data/benin-malanville.csv')

Summary Statistics & Missing-Value Report

In [17]:
# 1. Describe all numeric columns
print("--- Descriptive Statistics for Numeric Columns ---")
print(df.describe())

# 2. Calculate the number of missing values per column
print("\n--- Missing Value Counts per Column ---")
missing_counts = df.isna().sum()
print(missing_counts)

# 3. Identify columns with > 5% nulls
total_rows = len(df)
null_percentage_threshold = 0.05  # 5%

columns_with_high_nulls = missing_counts[missing_counts /
                                            total_rows > null_percentage_threshold].index.tolist()

print("\n--- Columns with > 5% Missing Values ---")
if columns_with_high_nulls:
    print(columns_with_high_nulls)
else:
    print("No columns have more than 5% missing values.")


--- Descriptive Statistics for Numeric Columns ---
                 GHI            DNI            DHI           ModA  \
count  525600.000000  525600.000000  525600.000000  525600.000000   
mean      240.559452     167.187516     115.358961     236.589496   
std       331.131327     261.710501     158.691074     326.894859   
min       -12.900000      -7.800000     -12.600000       0.000000   
25%        -2.000000      -0.500000      -2.100000       0.000000   
50%         1.800000      -0.100000       1.600000       4.500000   
75%       483.400000     314.200000     216.300000     463.700000   
max      1413.000000     952.300000     759.200000    1342.300000   

                ModB           Tamb             RH             WS  \
count  525600.000000  525600.000000  525600.000000  525600.000000   
mean      228.883576      28.179683      54.487969       2.121113   
std       316.536515       5.924297      28.073069       1.603466   
min         0.000000      11.000000       2.100000 

Outlier Detection & Basic Cleaning

In [ ]:
"""
Looks for missing values, potential outliers, and basic statistics
for the specified columns in the DataFrame.

Args:
    df (pd.DataFrame): The input DataFrame.
    columns_to_inspect (list): A list of column names to inspect.
"""
columns_to_inspect = ['GHI', 'DNI', 'DHI', 'ModA', 'ModB', 'WS', 'WSgust']
for col in columns_to_inspect:
    print(f"\n--- Column: {col} ---")

    # 1. Missing Values
    missing_count = df[col].isnull().sum()
    missing_percentage = (missing_count / len(df)) * 100
    print(f"Missing Value Count: {missing_count}")
    print(f"Missing Value Percentage: {missing_percentage:.2f}%")

    # 2. Basic Statistics
    print("\n--- Basic Statistics ---")
    print(df[col].describe())

    # 3. Outlier Detection (using IQR and Visualization)
    print("\n--- Outlier Detection (IQR) ---")
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers_iqr = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
    print(f"Number of potential outliers (IQR): {len(outliers_iqr)}")
    if not outliers_iqr.empty:
        print(f"Example outlier values:\n{outliers_iqr[col].head()}")

    # 4. Visualization (Box Plot for Outliers)
    plt.figure(figsize=(6, 4))
    sns.boxplot(x=df[col])
    plt.title(f"Box Plot of {col}")
    plt.xlabel(col)
    plt.show()

    # 5. Potential Incorrect Entries (based on domain knowledge - adjust as needed)
    print("\n--- Potential Incorrect Entries ---")
    if col in ['GHI', 'DNI', 'DHI', 'ModA', 'ModB']:
        # Allowing a small negative due to potential sensor noise
        negative_values = df[df[col] < -5][col]
        if not negative_values.empty:
            print(
                f"Number of potentially incorrect (significantly negative) values: {len(negative_values)}")
            print(f"Example negative values:\n{negative_values.head()}")
        else:
            print("No significantly negative values found.")
    elif col in ['WS', 'WSgust']:
        # Wind speed should generally be non-negative
        negative_ws = df[df[col] < 0][col]
        if not negative_ws.empty:
            print(
                f"Number of potentially incorrect (negative) wind speed values: {len(negative_ws)}")
            print(
                f"Example negative wind speed values:\n{negative_ws.head()}")
        else:
            print("No negative wind speed values found.")

    Computes Z-scores for specified columns and flags rows where the absolute
    Z-score is greater than 3.

In [21]:
def flag_zscore_outliers(df, columns):
    """
    Computes Z-scores for specified columns and flags rows where the absolute
    Z-score is greater than 3.

    Args:
        df (pd.DataFrame): The input DataFrame.
        columns (list): A list of column names to compute Z-scores for.

    Returns:
        pd.DataFrame: The DataFrame with a new boolean column 'is_zscore_outlier'
                      indicating if any of the specified columns have a |Z|>3.
    """
    df['is_zscore_outlier'] = False  # Initialize the flag column

    for col in columns:
        if col in df.columns and pd.api.types.is_numeric_dtype(df[col]):
            # Calculate Z-scores, handling potential NaN values
            z_scores = np.abs(stats.zscore(df[col], nan_policy='omit'))

            # Flag rows where the absolute Z-score is greater than 3 for the current column
            outlier_mask = z_scores > 3
            df.loc[outlier_mask, 'is_zscore_outlier'] = True
        else:
            print(
                f"Warning: Column '{col}' not found or is not numeric. Skipping Z-score calculation.")

    return df


# Specify the columns for Z-score calculation
zscore_columns = ['GHI', 'DNI', 'DHI', 'ModA', 'ModB', 'WS', 'WSgust']

# Assuming your DataFrame is named 'df'
df_with_zscore_flags = flag_zscore_outliers(df.copy(), zscore_columns)

# Display the first few rows with the outlier flag
print(df_with_zscore_flags[['Timestamp'] +
      zscore_columns + ['is_zscore_outlier']].head())

# You can also see how many outliers were flagged:
num_zscore_outliers = df_with_zscore_flags['is_zscore_outlier'].sum()
print(
    f"\nNumber of rows flagged as Z-score outliers (|Z| > 3): {num_zscore_outliers}")

          Timestamp  GHI  DNI  DHI  ModA  ModB   WS  WSgust  is_zscore_outlier
0  2021-08-09 00:01 -1.2 -0.2 -1.1   0.0   0.0  0.0     0.4              False
1  2021-08-09 00:02 -1.1 -0.2 -1.1   0.0   0.0  0.0     0.0              False
2  2021-08-09 00:03 -1.1 -0.2 -1.1   0.0   0.0  0.3     1.1              False
3  2021-08-09 00:04 -1.1 -0.1 -1.0   0.0   0.0  0.2     0.7              False
4  2021-08-09 00:05 -1.0 -0.1 -1.0   0.0   0.0  0.1     0.7              False

Number of rows flagged as Z-score outliers (|Z| > 3): 7740


In [22]:
def handle_missing_values(df, key_columns, imputation_strategy='median'):
    """
    Drops rows with missing values or imputes them using the median
    for the specified key columns.

    Args:
        df (pd.DataFrame): The input DataFrame.
        key_columns (list): A list of column names where missing values should be handled.
        imputation_strategy (str, optional): Strategy for handling missing values.
                                             'drop' to remove rows with any missing value in key columns.
                                             'median' to impute missing values with the median of the column.
                                             Defaults to 'median'.

    Returns:
        pd.DataFrame: The DataFrame with missing values handled in the key columns.
    """
    df_handled = df.copy()  # Work on a copy

    print(f"--- Handling Missing Values in Columns: {key_columns} ---")
    print(f"Initial number of rows: {len(df_handled)}")
    print("\nMissing values before handling:")
    print(df_handled[key_columns].isnull().sum())

    for col in key_columns:
        if col in df_handled.columns:
            if imputation_strategy.lower() == 'drop':
                initial_rows = len(df_handled)
                df_handled.dropna(subset=[col], inplace=True)
                rows_dropped = initial_rows - len(df_handled)
                print(
                    f"Dropped {rows_dropped} rows with missing values in column '{col}'.")
            elif imputation_strategy.lower() == 'median':
                median_val = df_handled[col].median()
                df_handled[col].fillna(median_val, inplace=True)
                print(
                    f"Imputed missing values in column '{col}' with median: {median_val:.2f}")
            else:
                print(
                    f"Warning: Unknown imputation strategy '{imputation_strategy}'. Skipping column '{col}'.")
        else:
            print(f"Warning: Key column '{col}' not found in DataFrame.")

    print(f"\nNumber of rows after handling missing values: {len(df_handled)}")
    print("\nMissing values after handling:")
    print(df_handled[key_columns].isnull().sum())

    return df_handled


# Specify the key columns where you want to handle missing values
key_columns_to_handle = ['GHI', 'DNI', 'DHI', 'ModA', 'ModB', 'WS', 'WSgust']

# Assuming your DataFrame is named 'df'

# Option 1: Drop rows with missing values in key columns
df_dropped_na = handle_missing_values(
    df.copy(), key_columns_to_handle, imputation_strategy='drop')
print("\nDataFrame after dropping missing values:")
print(df_dropped_na[key_columns_to_handle].head())

# Option 2: Impute missing values with the median in key columns
df_imputed_median = handle_missing_values(
    df.copy(), key_columns_to_handle, imputation_strategy='median')
print("\nDataFrame after imputing missing values with median:")
print(df_imputed_median[key_columns_to_handle].head())

--- Handling Missing Values in Columns: ['GHI', 'DNI', 'DHI', 'ModA', 'ModB', 'WS', 'WSgust'] ---
Initial number of rows: 525600

Missing values before handling:
GHI       0
DNI       0
DHI       0
ModA      0
ModB      0
WS        0
WSgust    0
dtype: int64
Dropped 0 rows with missing values in column 'GHI'.
Dropped 0 rows with missing values in column 'DNI'.
Dropped 0 rows with missing values in column 'DHI'.
Dropped 0 rows with missing values in column 'ModA'.
Dropped 0 rows with missing values in column 'ModB'.
Dropped 0 rows with missing values in column 'WS'.
Dropped 0 rows with missing values in column 'WSgust'.

Number of rows after handling missing values: 525600

Missing values after handling:
GHI       0
DNI       0
DHI       0
ModA      0
ModB      0
WS        0
WSgust    0
dtype: int64

DataFrame after dropping missing values:
   GHI  DNI  DHI  ModA  ModB   WS  WSgust
0 -1.2 -0.2 -1.1   0.0   0.0  0.0     0.4
1 -1.1 -0.2 -1.1   0.0   0.0  0.0     0.0
2 -1.1 -0.2 -1.1   0.0

/tmp/ipykernel_131130/2226578486.py:34: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_handled[col].fillna(median_val, inplace=True)
/tmp/ipykernel_131130/2226578486.py:34: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)'

    Exports the cleaned DataFrame to a CSV file in the specified directory.

In [23]:
def export_cleaned_data(df, country_name, output_dir='data'):
    """
    Exports the cleaned DataFrame to a CSV file in the specified directory.
    Args:
        df (pd.DataFrame): The cleaned DataFrame to export.
        country_name (str): The name of the country (used in the filename).
        output_dir (str, optional): The directory to save the CSV file.
                                     Defaults to 'data'.
    """
    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Create the filename
    filename = f"{country_name}_clean.csv"
    filepath = os.path.join(output_dir, filename)

    # Export the DataFrame to CSV
    # index=False prevents writing the DataFrame index to the CSV
    df.to_csv(filepath, index=False)

    print(f"Cleaned DataFrame for {country_name} exported to: {filepath}")
    print(
        f"Ensure the '{output_dir}/' directory is in your .gitignore file to avoid committing CSVs.")


# Example Usage:
# Assuming you have a cleaned DataFrame named 'df_cleaned' and the country is 'benin'
country = "benin"
cleaned_df = df.copy()  # Replace with your actual cleaned DataFrame

export_cleaned_data(cleaned_df, country)

Cleaned DataFrame for benin exported to: data/benin_clean.csv
Ensure the 'data/' directory is in your .gitignore file to avoid committing CSVs.
